In [7]:
import os
import re
import pandas as pd
data_path = ""
column_headers = ['Product', 'CO2 from raw materials', 'CO2 from energy supply', 'N2O & CH4 from production', 'CO2 from production', 'CO2 captured in product', 'Total (incl. captured CO2)', 'Total(excl. captured CO2)']

In [8]:
def read_txt_to_pd(text:str, column_headers) -> pd.DataFrame:
    start = 0
    while start < len(text):
        first = text[start:].find('0.')
        name = text[start:start+first]
        last_search = re.search('[a-zA-Z]', text[start+first:])
        try: last = last_search.span()[0]
        except: last = len(text)-start+first
        values = text[start+first:start+first+last-1].rstrip().replace(" ", ",")
        with open('temp.txt', 'a') as f:
            f.write(name.replace(',','').rstrip()+','+values+'\n')
        start = start+first+last

    emission_factors = pd.read_csv('temp.txt', header=None)
    os.remove('temp.txt')
    emission_factors.columns = column_headers
    return emission_factors

In [9]:
# Single value read
f = open(data_path+'all_fertilisers.txt', "r")
text = f.read().replace("\n", " ")
emission_factors = read_txt_to_pd(text, column_headers)

regional = pd.DataFrame()
for file in os.listdir(data_path+'emission_factor_txts/'):
    f = open(data_path+'emission_factor_txts/'+file)
    text = f.read().replace("\n", " ")
    fert = read_txt_to_pd(text, column_headers).rename(columns={'PRODUCT':'Region'})
    fert['PRODUCT'] = file.split('.tx')[0]
    regional = pd.concat((regional, fert), axis=0)

In [10]:
converted = emission_factors.copy()
converted['ifa_CO2e_100a_Direct Process factor'] = -converted['CO2 captured in product']
converted['ifa_CO2e_100a_Direct Utilities factor'] = converted['N2O & CH4 from production']+converted['CO2 from production']
converted = converted.rename(columns={'CO2 from raw materials':'ifa_CO2e_100a_Raw Material factor', 'CO2 from energy supply':'ifa_CO2e_100a_Indirect Utilities factor', 'Total(excl. captured CO2)':'ifa_CO2e_100a_CtOG factor'}).drop(columns=['Total (incl. captured CO2)', 'CO2 captured in product', 'CO2 from production', 'N2O & CH4 from production'])
converted['ifa_CO2e_100a_CtOG factor'] = converted['ifa_CO2e_100a_CtOG factor']+converted['ifa_CO2e_100a_Direct Process factor']

converted

,Product,ifa_CO2e_100a_Raw Material factor,ifa_CO2e_100a_Indirect Utilities factor,ifa_CO2e_100a_CtOG factor,ifa_CO2e_100a_Direct Process factor,ifa_CO2e_100a_Direct Utilities factor
0,Ammonia,0.000,0.395,2.297,-0.000,1.901
1,Ammonium nitrate (33.5-0-0) granulated,0.000,0.173,1.145,-0.000,0.971
2,Ammonium nitrate (33.5-0-0) prilled,0.000,0.166,1.112,-0.000,0.945
3,Ammonium nitrate sulphate,0.000,0.130,0.797,-0.000,0.667
4,Ammonium sulphate,0.000,0.101,0.563,-0.000,0.462
5,Calcium ammonium nitrate (27-0-0),0.014,0.142,0.950,-0.000,0.794
6,Calcium nitrate (15.5-0-0),0.000,0.089,0.636,-0.000,0.547
7,DAP (18-46-0),0.074,0.091,0.562,-0.000,0.398
8,Mixed acid process,0.082,0.082,0.595,-0.000,0.431
9,MAP (11-52-0),0.083,0.056,0.366,-0.000,0.227


In [35]:
converted.to_csv('../data/extracted/conversionFactors_from_IFA.csv')

In [5]:
## Add uncertainties

converted = pd.read_csv('../data/extracted/conversionFactors_from_IFA.csv', index_col=0)

conversion_uncertainty = 0.1

for col in converted.columns[1:]:
    converted[col.replace('factor','sigma')] = converted[col]*conversion_uncertainty

converted.to_csv('../data/extracted/conversionFactors_from_IFA_w_uncertainties.csv')